This introduction to logistic regression follows Chapter 5 of Stanford's Speech and Language Processing course. 

## Binary Logistic Regression

Recall the the Naive Bayes classifier assigns class $c$ to a document $d$, not by directly computing $P(c|d)$, but by computing a likelihood and a prior

$$
\hat{c} = \underset{c \in C}{\text{argmax}} \:\: \overbrace{P(d|c)}^{\text{likelihood}}\:\: \overbrace{P(c)}^{\text{prior}}
$$
Generative models (like Naive Bayes) use a likelihood term to express the probability of generating the document's features given that it was of class $c$. Discriminative models directly compute $P(c|d)$ to discriminate between potential classes, even if it couldn't generate an example of a given class. 


#### Components of a Probablistic Machine Learning Classifier
Naive Bayes and logistic regression both require a training set of $m$ input/ouput pairs $\left( x^{(i)},y^{(i)} \right)$, where the supserscript notation refers an inidivudal instance in the training set. The ML classification system has four parts: 
* Feature representation of input. Input $x^{(i)}$ is represented as a vector of features $[x_1,x_2,...,x_n]$. $x^{(j)}_i$ represents feature $i$ for input $x^{(j)}$ , synonymous with $x_i$, $f_i$, and $f_i(x)$. 
* Classification function to calculate estimated class $\hat{y}$ through $p(y|x)$, using sigmoid and softmax tools. 
* Objective function for learning to minimizes error on training examples. In this case, the cross-entropy loss functions.
* An algorithm to optimize the objective function. In this case, stochastic gradient descent (SGD). 

#### Sigmoid Function
The sigmoid function helps classifiers make a binary decision about the class of a new input. For the input observation $x$, represented by feature vector $[x_1, x_2,...,x_n]$, the classifier output $y$ can be 1 (observation is a member of class) or 0. To calculate $P(y=1|x)$, logistic regression learns (from a training set) a vector of weights and a bias term. Each weight $w_i$ is associated with one of the input features $x_i$, reflecting how important the input feature is to the classification decision, and can be positive or negative. The bias term (intercept) is added to the weighted inputs. 

After learning these values in training, the classifier decides on a test instance by first multiplying each $x_i$ by its weight $w_i$, summing these values, and adding the bias term. $z$, the weighted sum of evidence for this class, can be expressed using dot product notation, using boldface notation to represent vectors: 
$$
z = \left( \sum_{i=1}^n w_i x_i \right) + b = \textbf{w} \cdot \textbf{x} + b
$$
To convert $z$ to a legal probability between 0 and 1, we pass it through the sigmoid function $\sigma(z)$ (resembling an s, and also called the logistic regression). 
$$
\sigma(z) = \frac{1}{1 + e^{-z}} = \frac{1}{1 + \exp(-z)}
$$
We can then calculate probabilities of the two cases as follows (which sum to 1):
$$
P(y=1) = \sigma(\textbf{w} \cdot \textbf{x} + b) = \frac{1}{1 + \exp(-(\textbf{w} \cdot \textbf{x} + b))}
$$
$$
P(y=0) = 1 - \sigma(\textbf{w} \cdot \textbf{x} + b) = \frac{\exp(-(\textbf{w} \cdot \textbf{x} + b))}{1 + \exp(-(\textbf{w} \cdot \textbf{x} + b))}
$$
Setting 0.5 as the decision boundary, for a test instance $x$, we say "yes" if $P(y=1|x) > 0.5$, and " no" otherwise. 

#### Features
Features are traditionally designed by exaimining the training dataset. When doing this, it's important to consider feature interaction (expressing the prediction as a sum of feature effects when the effect of one feature depends on the value of the other). To avoid extensive human effort of feature design, more recent NLP work has incorporated representation learning: ways to learn features in an automatic and unsupervised manner from the input. 

When different features have different ranges, it's common to scale them by standariziing input values, resulting in a zero mean and standard deviation of one. That is, if $\mu_i$ is the mean of the values of feature $\textbf{x}_i$ across $m$ observations in the input dataset, and $\sigma_i$ is the standard deviation of these values, we replace each feature $\textbf{x}_i$ with $\textbf{x}^\prime_i$:
$$
\mu_i = \frac{1}{m} \sum_{j=1}^m x_i^{(j)} \quad \text{and} \quad \sigma_i \sqrt{\frac{1}{m} \sum_{j=1}^m \left(x_i^{(j)} - \mu_i \right)^2}
$$
$$
\textbf{x}^\prime_i = \frac{\textbf{x}_i - \mu_i}{\sigma_i}
$$
Alternatively, we can normalize the input features, creating a range between 0 and 1: 
$$
\textbf{x}^\prime_i  = \frac{\textbf{x}_i - \min(\textbf{x}_i)}{\max(\textbf{x}_i) - min(\textbf{x}_i)}
$$
Both data scaling options help compare values across features. 

#### Vectorization
We need to process an entire test set with $m$ test examples needing classification. Each test example $x^{(i)}$ has a feature vector $\textbf{x}^{(i)}$, for $1 \leq i \leq m$. We could calculate $\hat{y}^{(i)}$ with a for-loop:
$$
\begin{align*}
\textbf{foreach} \:\: & x^{(i)} \text{ in input } [x^{(1)}, x^{(2)},...,x^{(m)}] \\
&y^{(i)} = \sigma(\textbf{w}\cdot\textbf{x}^{(i)}+b)
\end{align*}
$$
For the first 3 test examples,
$$
P(y^{(1)} = 1|x^{(1)}) = \sigma(\textbf{w}\cdot\textbf{x}^{(1)}+b)
$$
$$
P(y^{(2)} = 1|x^{(2)}) = \sigma(\textbf{w}\cdot\textbf{x}^{(2)}+b)
$$
$$
P(y^{(3)} = 1|x^{(3)}) = \sigma(\textbf{w}\cdot\textbf{x}^{(3)}+b)
$$
Seeing a pattern, we can use matrix arithemtic to make the process more efficient. We will include all input feature vectors for input $x$ (remembering that $x$ has $m$ test examples) into a single input matrix $\textbf{X}$. Each row $i$ corrresponds to the feature vector for input example $x^{(i)}$. If each exaample has $f$ features, we construct a $m \times f$ matrix: 
$$
\textbf{X} = \begin{bmatrix}
x^{(1)}_1 & x^{(1)}_2 & \dots & x^{(1)}_f \\
x^{(2)}_1 & x^{(2)}_2 & \dots & x^{(2)}_f \\
\vdots    & \vdots    & \ddots& \vdots    \\
x^{(m)}_1 & x^{(m)}_2 & \dots & x^{(m)}_f \\
\end{bmatrix}_{m \times f}
$$

Now, introducing $\textbf{b}$ as a vector of length $m$ with the scalar bias terms repeated $m$ times and $\hat{\textbf{y}}^{(i)}$, 
$$
\textbf{b} = \begin{bmatrix}
b \\ b \\ \vdots \\ b
\end{bmatrix} \quad \text{and} \quad {\textbf{y}} = \begin{bmatrix}
\hat{y}^{(1)} \\ \hat{y}^{(2)} \\ \vdots \\ \hat{y}^{(m)}
\end{bmatrix}
$$

Now, all outputs can be computed with a single matrix multiplication followed by an addition: 
$$
\textbf{y = Xw + b}
$$
Note that this computes the same thing as the for-loop. Also note that $\textbf{y}$ is $(m \times 1)$, $\textbf{X}$ is $(m \times f)$, $\textbf{w}$ is $(f \times 1)$, and $\textbf{b}$ is $(m \times 1)$. Also note that logistic regression works generally works better on larger datasets or documents. In settings with many correlated features, logistics regression will outperform Naive Bayes, due to the independence assumption. 

## Multinomial Logistic Regression
For classification problems with more than two classes, we must instead use multinomial logistic regression (also referred to as softmax regression). We want to label each observation with a class $k$ from a set of $K$ classes, assuming that only one of these classes is correct (hard classification). 

The output $\textbf{y}$ for each input $\textbf{x}$ will be a vector of lenght $K$. If class $c$ is correct, $\textbf{y}_c = 1$, and all other elements of $\textbf{y}$ are set to 0. That is, $\textbf{y}_c = 1$ and $\textbf{y}_j = 0 \:\: \forall j \neq c$, a one-hot vector. The classifier will produce an estimate vector $\hat{\textbf{y}}$, containing $p(\textbf{y}_k = 1 | \textbf{x})$ for each class. 

#### Softmax Function
The softmax function, a generalization of the sigmoid, is used to calculate $p(y_k =1|\textbf{x})$. We input a vector $\textbf{z} = [\textbf{z}_1, \textbf{z}_2,...,\textbf{z}_K]$ of $K$ arbitrary values, and the function maps them to a probability distribution ranging from 0 to 1, with all values summing to 1. 
$$
\text{softmax}(\textbf{z}_i) = \frac{\exp(\textbf{z}_i)}{\sum_{j=1}^K \exp(\textbf{z}_j)} \quad 1 \leq i \leq K
$$
The denominator normalizes all values into probabilities. Like the sigmoid, if one input is much larger than the others, it pushes the probability towards 1, suppressing the smaller inputs. Note that the input and output of softmax are both vectors: 
$$
\text{softmax}(\textbf{z}) = \left[\frac{\exp(\textbf{z}_1)}{\sum_{i=1}^K \exp(\textbf{z}_i)}, \frac{\exp(\textbf{z}_2)}{\sum_{i=1}^K \exp(\textbf{z}_i)}, ... , \frac{\exp(\textbf{z}_K)}{\sum_{i=1}^K \exp(\textbf{z}_i)} \right]
$$

Applying this to logistic regression, the input will (similarly to the sigmoid) be the dot product of weight vector $\textbf{w}$ and input vector $\textbf{x}$, plus a bias. But, we now need to seperate weight vectors $\textbf{w}_k$ and bias $b_k$ for each of the $K$ classes. The probability of each of each class is computed as
$$
p(\textbf{y}_k = 1|\textbf{x}) = \frac{\exp(\textbf{w}_k \cdot \textbf{x} + b_k)}{\sum_{j=1}^K \exp(\textbf{w}_j \cdot \textbf{x} + b_j)}
$$
Once again instead of computing each output seperateltly, we can make the process more efficient using matrices. Let the weight matrix $\textbf{W}$ represent the set of $K$ weight vectors, with each row $k$ corresponding to the vector of weights $w_k$ (how much those weights impact a given class $k$). $\textbf{W}$ is $K \times f$, where $K$ is the number of possible output classes and $f$ is the number of input features. The bias vector $\textbf{b}$ now has one value for each of the $K$ input features (where before, it was constant since we simply compared to our decision boundary of 0.5). With this representation, we can compute the the vector of output probabilities for each of the $K$ classes $\hat{\textbf{y}}$ as follows:
$$
\hat{\textbf{y}} = \text{softmax}(\textbf{Wx} + \textbf{b})
$$
For example, for the first class, $\hat{\textbf{y}}_1$ is given by $\textbf{w}_1 \cdot \textbf{x} + b_1$, put through softmax. Noting some key differences between binary and multinomial logistic regression, we now have more rows in our weight and bias matrices. The input is still a feature vector, but the output is now a vector (softmax), not a scalar (sigmoid). 

#### Cross-Entropy Loss Function
To learn the parameters of the model (in this case, weights and bias), we must first introduce a cost function. We will then iteratively update the weights to minimize this through an optimization, in this case through stochastic gradient descent. This will be introduced through the binary case, and then generalized to the multinomial case. 

The loss function $L(\hat{y},y)$ describes, for an observation $x$, how close the classifier output ($\hat{y} = \sigma(\textbf{w}\cdot\ \textbf{x} + b)$) is to the correct output ($y$, either 0 or 1). We use a loss function that prefers the correct class labels of training examples to more likely. That is, we choose weights and bias that maximize the log probability of the true $y$ labels in the training idea, a process called conditional maximum likelihood estimation. The resulting cross-entropy loss function is the negative log likelihood loss. In other words, the cross-entropy loss function quantifies the difference between the true labels and predictions. After computing the loss across predictions, we penalize the model for predictions for off from "true" labels, adjusting the parameters to maximize log likelihood. 

For an observation $x$, we hope to learn weights that maximize the probability of the correct label $p(y|x)$. Since we have two discrete outcomes (0 and 1), we're dealing with a Bernoulli distriubution, and can represent $p(y|x)$ as follows to derive the loss function: 
$$
p(y|x) = \hat{y}^y (1 - \hat{y})^{1-y}
$$
Note that this simplifies to $\hat{y}$ when $y = 1$ and  $1 - \hat{y}$ when $y = 0$. By rearranging, flipping the sign, and plugging in our definition of $\hat{y} = \sigma(\textbf{w} \cdot \textbf{x} + b)$,  we create a cross-entropy loss function $L_{CE}$ to be minimized:
$$
\log p(y|x) = \log \left[ \hat{y}^y (1 - \hat{y})^{1-y} \right]
$$
$$
\log p(y|x) = \log \hat{y}^y + \log (1 - \hat{y})^{1-y} 
$$
$$
\log p(y|x) = y \log \hat{y} + (1-y)\log (1 - \hat{y}) 
$$
$$
L_{CE} = -\log p(y|x) = -\left[y \log \hat{y} + (1-y)\log (1 - \hat{y}) \right]
$$
$$\boxed{\boxed{
L_{CE} = -\left[y \log \sigma(\textbf{w} \cdot \textbf{x} + b) + (1-y)\log (1 - \sigma(\textbf{w} \cdot \textbf{x} + b)) \right]}}
$$

#### Gradient Descent 
Having quantified the cross-entropy between true probability $y$ and estimated distribution $\hat{y}$, we now have a function to minimize. Our loss function is parameterized by weights, refered to as $\theta$ (in this case, $\theta = w,b$). The goal is now to find the set of which minimize the average of the loss function across all training examples: 
$$
\hat{\theta} = \underset{\theta}{\text{argmin}} \frac{1}{m} \sum_{i=1}^m \overbrace{L_{CE}  \left( \overbrace{f(x^{(i)}; \theta )}^{\text{prediction}}, \overbrace{y^{(i)}}^{\text{label}}\right)}^{\text{one instance}}
$$
This function is convex (with only minimum, there are no local minima for gradient descent to get stuck in). By finding the gradient of the loss function and moving in the opposite direction with a magnitude weighted by learning rate $\eta$, we find this minimum. For each variable $w_i$, we find the partial derivative $\frac{\partial L}{\partial w_i}$. Formally, and representing $\hat{y}$ as $f(x;\theta)$ to emphasis the predictions's dependance on parameters, 
$$
\nabla L(f(x;\theta),y) = \begin{bmatrix}
\frac{\partial}{\partial w_1} L(f(x;\theta),y) \\
\frac{\partial}{\partial w_2} L(f(x;\theta),y) \\
\vdots \\
\frac{\partial}{\partial w_n} L(f(x;\theta),y) \\
\frac{\partial}{\partial b} L(f(x;\theta),y) \\
\end{bmatrix}
$$
The final equation for iteratively updating parameters $\theta$ based on this gradient is
$$\boxed{\boxed{
\theta^{t+1} = \theta^t - \eta \nabla L(f(x;\theta),y)}}
$$
Recall the cross-entropy loss function 
$$
L_{CE}(\hat{y},y) = -\left[y \log \sigma(\textbf{w} \cdot \textbf{x} + b) + (1-y)\log (1 - \sigma(\textbf{w} \cdot \textbf{x} + b)) \right]
$$
Finding the derivative of our function for observation vector $x$, 
$$
 \frac{\partial L_{\mathrm{CE}}}{\partial \mathbf{w}_j} =\frac{\partial}{\partial \mathbf{w}_j}-[y \log \sigma(\mathbf{w} \cdot \mathbf{x}+b)+(1-y) \log (1-\sigma(\mathbf{w} \cdot \mathbf{x}+b))] 
$$
$$
 \frac{\partial L_{\mathrm{CE}}}{\partial \mathbf{w}_j} =-\left[\frac{\partial}{\partial \mathbf{w}_j} y \log \sigma(\mathbf{w} \cdot \mathbf{x}+b)+\frac{\partial}{\partial \mathbf{w}_j}(1-y) \log [1-\sigma(\mathbf{w} \cdot \mathbf{x}+b)]\right]
$$
$$
\frac{\partial L_{\mathrm{CE}}}{\partial \mathbf{w}_j}=-\frac{y}{\sigma(\mathbf{w} \cdot \mathbf{x}+b)} \frac{\partial}{\partial \mathbf{w}_j} \sigma(\mathbf{w} \cdot \mathbf{x}+b)-\frac{1-y}{1-\sigma(\mathbf{w} \cdot \mathbf{x}+b)} \frac{\partial}{\partial \mathbf{w}_j} 1-\sigma(\mathbf{w} \cdot \mathbf{x}+b)
$$
$$
\frac{\partial L_{\mathrm{CE}}}{\partial \mathbf{w}_j}=-\left[\frac{y}{\sigma(\mathbf{w} \cdot \mathbf{x}+b)}-\frac{1-y}{1-\sigma(\mathbf{w} \cdot \mathbf{x}+b)}\right] \frac{\partial}{\partial \mathbf{w}_j} \sigma(\mathbf{w} \cdot \mathbf{x}+b)
$$
$$
\frac{\partial L_{\mathrm{CE}}}{\partial \mathbf{w}_j} =-\left[\frac{y-\sigma(\mathbf{w} \cdot \mathbf{x}+b)}{\sigma(\mathbf{w} \cdot \mathbf{x}+b)[1-\sigma(\mathbf{w} \cdot \mathbf{x}+b)]}\right] \sigma(\mathbf{w} \cdot \mathbf{x}+b)[1-\sigma(\mathbf{w} \cdot \mathbf{x}+b)] \frac{\partial(\mathbf{w} \cdot \mathbf{x}+b)}{\partial \mathbf{w}_j} 
$$
$$
\frac{\partial L_{\mathrm{CE}}}{\partial \mathbf{w}_j} =-\left[\frac{y-\sigma(\mathbf{w} \cdot \mathbf{x}+b)}{\sigma(\mathbf{w} \cdot \mathbf{x}+b)[1-\sigma(\mathbf{w} \cdot \mathbf{x}+b)]}\right] \sigma(\mathbf{w} \cdot \mathbf{x}+b)[1-\sigma(\mathbf{w} \cdot \mathbf{x}+b)] \mathbf{x}_j
$$
$$
\frac{\partial L_{\mathrm{CE}}}{\partial \mathbf{w}_j} =-[y-\sigma(\mathbf{w} \cdot \mathbf{x}+b)] \mathbf{x}_j 
$$
$$
\frac{\partial L_{\mathrm{CE}}}{\partial \mathbf{w}_j} =[\sigma(\mathbf{w} \cdot \mathbf{x}+b)-y] \mathbf{x}_j
$$
$$\boxed{\boxed{
\frac{\partial L_{CE}(\hat{y},y)}{\partial w_j} = (\hat{y} - y)\textbf{x}_j = -(y - \hat{y} )\textbf{x}_j}}
$$

#### Stochastic Gradient Descent

The SGD function is a function that returns $\theta$. After defining loss function $L$ and function $f$ parameterized by $\theta$, it takes inputs of $x$ (the set of training inputs) and $y$ (the set of corresponding labels). 

For each training tuple $\left( x^{(i)}, y^{(i)}\right)$, it first calculates estimated ouput $\hat{y}^{(i)} = f(x^{(i)};\theta)$. It then computes how far the estimate is from the label through loss $L(\hat{y}^{(i)},y^{(i)})$. It then calculates the gradient of the loss function with respect to each model parameter as $\nabla_\theta L \left( f(x^{(i)};\theta),y^{(i)}\right)$, adjusting values using this value weighted by the learning rate. We repeat this process until convergence or a halt in progress. 

Each partial derivative measures the magnitude of the feature value and the correctness of the model. When the model is incorrect, the weights corresponding to the largest current feature will be changed the most. One can also start with a higher learning rate and decrease it through iterations, taking smaller steps as the minimum is approached (the notation $\eta_k$ is useful for this). 

By choosing a single random example and modifying the weights to improve performance on that single sample, the resulting movement can be choppy. It's common to instead compute gradients over batches of training instances. In batch training, the entire dataset is used for every movement, a computationally expensive and precise approach. Mini-batch training is a compromise, where we train on a group of $m$ examples. These mini-batches can be vectoried and computed at once in parellel. Extending the cross-entropy loss for one example to an entire mini batch and assuming the examples are independent, 
$$
\log p (\text{training examples}) = \underbrace{\log \prod_{i=1}^m p \left(  y^{(i)} | x^{(i)} \right)}_{\text{how well model predicts entire batch}}
$$
$$
\log p (\text{training examples}) = \underbrace{\sum_{i=1}^m \log p \left(  y^{(i)} | x^{(i)} \right)}_{\text{log of product is sum of individual logs}}
$$
$$
\log p (\text{training examples}) = \underbrace{- \sum_{i=1}^m L_{CE} \left(  \hat{y}^{(i)} | y^{(i)} \right)}_{\text{recall that } L_{CE} = -\log p(y|x)}
$$
The cost function for the mini-batch containing $m$ examples is now the average loss:
$$
Cost(\hat{y},y) = \frac{1}{m} \sum_{i=1}^m L_{CE}\left(  \hat{y}^{(i)},{y}^{(i)} \right)
$$
$$
Cost(\hat{y},y) = - \frac{1}{m} \sum_{i=1}^m y^{(i)} \log \sigma(\textbf{w} \cdot \textbf{x}^{(i)} + b) + (1-y^{(i)})\log \left(1 - \sigma(\textbf{w} \cdot \textbf{x}^{(i)} + b)\right)
$$
The gradient of the mini-batch is the average of the individual gradients, which can be vectorized. The matrix $\textbf{X}$  of size $m \times f$ represents the inputs of the batch, and vector $\textbf{y}$ of size $m \times 1$ represents the correct outputs (labels):
$$
\frac{\partial Cost(\hat{y},y)}{\partial w_j} = \frac{1}{m} \sum_{i=1}^m (\hat{y}^{(i)} - y^{(i)})\textbf{x}_j^{(i)}
$$
$$
\frac{\partial Cost(\hat{y},y)}{\partial w_j} = \frac{1}{m} \sum_{i=1}^m \left[ \sigma(\textbf{w}\cdot \textbf{x}^{(i)}+b) - y^{(i)} \right] \textbf{x}_j^{(i)}
$$
$$
\frac{\partial Cost(\hat{y},y)}{\partial \textbf{w}} = \frac{1}{m} (\hat{\textbf{y}} - \textbf{y})^T \textbf{X}
$$
$$
\frac{\partial Cost(\hat{y},y)}{\partial \textbf{w}} = \frac{1}{m} (\sigma(\textbf{Xw} + \textbf{b})- \textbf{y})^T \textbf{X}
$$

#### Regularization
If we calibrate learning weights to perfectly match the training data, we may run into the challenge of overfitting. An overfitten model struggles to generalize well from training data to the unseen test set. To avoid this, we must introduce a regularization term. Recall how the objective function was initially defined: 
$$
\hat{\theta} = \underset{\theta}{\text{argmin}} \frac{1}{m} \sum_{i=1}^m \overbrace{L_{CE}  \left( \overbrace{f(x^{(i)}; \theta )}^{\text{prediction}}, \overbrace{y^{(i)}}^{\text{label}}\right)}^{\text{one instance}}
$$
Removing the $\frac{1}{m}$ (doesn't affect argmax) and maximizing log probability instead of minimizing loss, 
$$
\hat{\theta} = \underset{\theta}{\text{argmax}} \log P  \left( y^{(i)}, y^{(i)} \right) - \alpha R(\theta)
$$
$R(\theta)$ penalizes large weights, favoring a slightly worse fit with smaller weights over a perfect fit with large weights. This can be computed in two ways. Firstly, L2 regulariization is a quadratic function of weight values. The L2 norm $||\theta||_2$ is the Euclidean distance of the vector $\theta$ from the origin. If $\theta$ has $n$ weights, 
$$
R(\theta) = ||\theta||_2^2 = \sum_{j=1}^n \theta_j^2 \Longrightarrow \hat{\theta} = \underset{\theta}{\text{argmax}} \log P  \left( y^{(i)}, y^{(i)} \right) - \alpha \sum_{j=1}^n \theta_j^2
$$
L1 regularization is a linear function of weight values, the sum of the absolute values of the weights ("Manhattan distance"):
$$
R(\theta) = ||\theta||_1 = \sum_{i=1}^n |\theta_i| \Longrightarrow \hat{\theta} = \underset{\theta}{\text{argmax}} \log P  \left( y^{(i)}, y^{(i)} \right) - \alpha \sum_{i=1}^n |\theta_i|
$$
L1 regularization is called lasso regression, and L2 is called ridge regression. L2 is easier to optimize (it's derititivae is simpler) and prefers weight vectors with many small weights. L1 prefers some larger weights but more weights set to zero. L2 regularization can be thought of as a Gaussian distribution of weights with a mean $\mu = 0$. Using a Gaussain prior on weights, we say that they prefer a value of 0. A Gaussian for weight $\theta_j$ is
$$
\frac{1}{\sqrt{2 \pi \sigma_j^2}} \exp \left( - \frac{(\theta_j - \mu_j)^2}{2\sigma_j^2}  \right)
$$
Multiplying each weight by this Gaussian prior, moving to log space, and setting $\mu = 0$ and $2\sigma^2 = 1$, we find an equation of the same form:
$$
\hat{\theta} = \underset{\theta}{\text{argmax}} \prod_{i=1}^m P  ( y^{(i)}, y^{(i)}) \times \prod_{j=1}^n \frac{1}{\sqrt{2 \pi \sigma_j^2}} \exp \left( - \frac{(\theta_j - \mu_j)^2}{2\sigma_j^2}  \right)
$$
$$
\hat{\theta} = \underset{\theta}{\text{argmax}} \sum_{i=1}^m  \log P  ( y^{(i)}, y^{(i)}) - \alpha \sum_{j=1}^n \theta_j^2
$$

#### Learning in Multinomial Logistic Regression
Recall that cross-entropy loss for binary logistic regression is:
$$
L_{CE} = -\log p(y|x) = -\left[\overbrace{y \log \hat{y}}^{\text{term 1}} + \overbrace{(1-y)\log (1 - \hat{y})}^{\text{term 2}} \right]
$$
This has two terms, with one being non-zero when $y=1$ and the other being non-zero when $y=0$. We must generalize thiss to $K$ terms, using one-hot vectors. The true label $\textbf{y}$ contains $K$ elements, all of which are zero except for $\textbf{y}_c = 1$ for the correct class $c$. The classifier produces an estimate vector $\hat{\textbf{y}}$, with each $\textbf{y}_k$ representing the estimate for $P(\textbf{y}_k = 1 | \textbf{x})$. For a single example $\textbf{x}$, we can generalize the loss function to the sum of the logs of the the $K$ output classes, each weighted by their corresponding probability $\textbf{y}_k$. 
$$
L_{CE}(\hat{\textbf{y}}_k, \textbf{y}) = -\sum_{k=1}^K \textbf{y}_k \log \hat{\textbf{y}}_k
$$
Since this is a one-hot vector, $y_j = 0 \: \forall j \neq c$, where $c$ is the true class. 
$$
L_{CE}(\hat{\textbf{y}}_k, \textbf{y}) = -\log \hat{\textbf{y}}_c
$$
$$
L_{CE}(\hat{\textbf{y}}_k, \textbf{y}) = -\log \hat{p}(\textbf{y}_c = 1 | \textbf{x})
$$
$$
L_{CE}(\hat{\textbf{y}}_k, \textbf{y}) = -\log \frac{\exp(\textbf{w}_c \cdot \textbf{x} + b_c)}{\sum_{j-1}^K \exp(\textbf{w}_j\cdot \textbf{x} + b_j)}
$$
Therefore, cross-entropy loss is the log of the output probability corresponding to the correct class, refered to as negative log loikelhood loss (small when guess is close to 1, larger when guess is closer to 0, and negative so range is 0 to 1). The gradient, following an approach similar to the derivation earlier, comes out to be: 
$$
\frac{\partial L_{CE}}{\partial \textbf{w}_{k,i}} = - ({\textbf{y}}_k - \hat{\textbf{y}}_k)\textbf{x}_i
$$
$$
\frac{\partial L_{CE}}{\partial \textbf{w}_{k,i}} = - ({\textbf{y}}_k - p(\textbf{y}_k = 1 | \textbf{x}))\textbf{x}_i
$$
$$
\frac{\partial L_{CE}}{\partial \textbf{w}_{k,i}} = - \left({\textbf{y}}_k -  \frac{\exp(\textbf{w}_k \cdot \textbf{x} + b_k )}{\sum_{j=1}^K \exp(\textbf{w}_j \cdot \textbf{x} + b_j ) } \right)\textbf{x}_i
$$